In [ ]:
## This script will be used to create the confusion matrix
## as well as create some performance efficiency metrics

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 

In [ ]:
#Stim Trigs (ID)
#1 (Crying)
#6 (Unhappy)
#7 (Neutral)
#8 (Happy)

#Response Trigs (ID)
#2 ("A")
#3 ("S")
#4 ("K")
#5 ("L")

#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
## Run a test example here
df = pd.read_csv("/home/arosen/Documents/bbmcPersonal/eegBehavioralData/ID/250-26#_Parent_Identify_ICAb.evt", sep='\t', header=0)

In [3]:
## First remove all 628 values from the df
df = df[-df["TriNo"].isin(["628"])]

In [4]:
## Now isolate the stimuli presentation from the answers
df_stim = df[df['TriNo'].isin(['1', '6', '7', '8'])]
df_stim.head(5)

,Tmu,Code,TriNo,Comnt
2,4867000,1,7,Trigger - DIN8
4,6882000,1,6,Trigger - DI64
7,10730000,1,7,Trigger - DIN8
9,12528000,1,1,Trigger - DI16
12,14460000,1,6,Trigger - DI64


In [10]:
%qtconsole

In [ ]:
## Do the same for the reponses
df_resp = df[df['TriNo'].isin(['2', '3', '4', '5'])]
df_resp.head(5)

In [69]:
## Now I need to isolate the values that are inbetween intervals
## every stim should have 1 resp.. but there may be two for some
## This will be done in a for loop
## 118-314_OutResponse.csv 146

## and I will print the values that have multiple responses
## then I will have to take the second or first.... to be determined
row_vals = df_stim.shape[0]
## Now create an output data frame for everything to land into
d = pd.DataFrame([])
for x in range(row_vals):
    ## First identify the range
    lower_val = df_stim['Tmu         '].iloc[x]
    if x == df_stim.shape[0]:
        upper_val = df_stim['Tmu         '].iloc[x] + 1500000
    else:
        upper_val = df_stim['Tmu         '].iloc[x+1]
    ## Now check to see if the values are inbetween the interval in the resp
    if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
        print("Case at " ,x)
        ## Grab the last response
        row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
        #row_index = 0
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
    
    elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
        print("Null response @ ", x)
        out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
        out_row = pd.DataFrame(out_row)
        out_row.columns = df_stim.columns
    
    ## Now attach these all to the output
    else:
        out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
        
    input_stim = df_stim.iloc[x]
    d = d.append(input_stim)
    d = d.append(out_row)


Null response @  1
Case at  6
Null response @  7
Case at  8
Case at  12
Case at  14
Null response @  15
Null response @  19
Null response @  23
Case at  24
Case at  40
Case at  54
Case at  62
Case at  66
Case at  80
Case at  85
Case at  106
Null response @  115
Case at  120
Case at  155
Null response @  171
Case at  182
Null response @  186
Case at  201
Case at  234
Case at  269
Case at  288
Case at  302
Case at  347


IndexError: single positional indexer is out-of-bounds

In [ ]:
## Now grab all of the even values - these are the stimuli
# We are going through this for two reasons:
# 1. AFGR is terrible at python programming
# 2. We want to get the differece in presentation and answer
#  to do 2 I am going to attach these togerther and calculate the differnece
df_stim = d[d['TriNo'].isin(['1', '6', '7', '8'])]
df_ans = d[-d['TriNo'].isin(['1', '6', '7', '8'])]
df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)

In [ ]:
## Now calculate the difference in time
df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns']

In [ ]:
## Now create the difference between timeStim and TimeAns
df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

In [ ]:
## Now create a confusion matrix
#print(metrics.classification_report(df_c['TriNoStim'], df_c['TriNoAns']))

In [93]:
#!/usr/bin/env python
# coding: utf-8

# In[98]:


## This script will be used to create the confusion matrix
## as well as create some performance efficiency metrics

##  Acouple of notes I need to be aware of:
## 1. 628 is a nonsense code - I can remove these rows
## 2. I need to find a good way to protect against corrected answers
##   a. The stimuli are given a random interval any where between 1 and 1.5 seconds
##   in order to protect against this I need to find the windows between presentations
## 3. The goal is to determine how efficient these folks are at processing - so I will use the corrected answer
## it changes the answer 


# In[243]:


#Stim Trigs (ID)
#1 (Crying)
#6 (Unhappy)
#7 (Neutral)
#8 (Happy)

#Response Trigs (ID)
#2 ("A")
#3 ("S")
#4 ("K")
#5 ("L")

#Stim Trigs (Care)
#3 (Crying)
#4 (Unhappy)
#5 (Neutral)
#6 (Happy)

#Response Trigs (Care)
#1 ("L Shift")
#2 ("R Shift")


# In[244]:


import glob, os
import pandas as pd
import numpy as np
from sklearn import metrics


# In[245]:


## Run a test example here
os.chdir("/home/arosen/Documents/bbmcPersonal/eegBehavioralData/ID/")
for df in glob.glob("*.evt"):
    print(df)
    ## Now create the output name while we have the string in memory
    out_name = "/home/arosen/Documents/bbmcPersonal/eegBehavioralData/ID_mod/" + df.replace("#", "").partition("_")[0] + "_OutResponse.csv"
    df = pd.read_csv(df, sep='\t', header=0)

    
    
    ## First remove all 628 values from the df
    df = df[-df["TriNo"].isin(["628"])]
    
    
    # In[247]:
    
    
    ## Now isolate the stimuli presentation from the answers
    df_stim = df[df['TriNo'].isin(['1', '6', '7', '8'])]
    df_stim.head(5)
    
    
    # In[248]:
    
    
    ## Do the same for the reponses
    df_resp = df[-df['TriNo'].isin(['1', '6', '7', '8'])]
    df_resp.head(5)
    
    
    # In[249]:
    
    
    ## Now I need to isolate the values that are inbetween intervals
    ## every stim should have 1 resp.. but there may be two for some
    ## This will be done in a for loop
    ## and I will print the values that have multiple responses
    ## then I will have to take the second or first.... to be determined
    row_vals = df_stim.shape[0]
    ## Now create an output data frame for everything to land into
    d = pd.DataFrame([])
    for x in range(row_vals):
        ## First identify the range
        lower_val = df_stim['Tmu         '].iloc[x]
        if x == df_stim.shape[0]-1:
            upper_val = df_stim['Tmu         '].iloc[x] + 1500000
            print("Final Row")
        else:
            upper_val = df_stim['Tmu         '].iloc[x+1]
        ## Now check to see if the values are inbetween the interval in the resp
        if df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] > 1:
            #print("Case at " ,x)
            ## Grab the last response
            row_index = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] - 1
            #row_index = 0
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].iloc[row_index]
        
        elif df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0] == 0:
            #print("Null response @ ", x)
            out_row = {'col1': [lower_val], 'col2': [np.nan], 'col3': [np.nan], 'col4': [np.nan]}
            out_row = pd.DataFrame(out_row)
            out_row.columns = df_stim.columns
        
        ## Now attach these all to the output
        else:
            out_row = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)]
            
        input_stim = df_stim.iloc[x]
        input_stim['numOfResponses'] = df_resp[df_resp['Tmu         '].between(lower_val, upper_val)].shape[0]
        ## Now add a column with the number of responses
        d = d.append(input_stim)
        d = d.append(out_row)


    ## Now grab all of the even values - these are the stimuli
    # We are going through this for two reasons:
    # 1. AFGR is terrible at python programming
    # 2. We want to get the differece in presentation and answer
    #  to do 2 I am going to attach these togerther and calculate the differnece
    df_stim = d[d['TriNo'].isin(['1', '6', '7', '8'])]
    df_ans = d[-d['TriNo'].isin(['1', '6', '7', '8'])]
    df_c = pd.concat([df_stim.reset_index(drop=True), df_ans.reset_index(drop=True)], axis=1)


    ## Now calculate the difference in time
    df_c.columns = ['Index','CommentStim','TimeStim','TriNoStim','noOfResposesStim','CodeAns','CommentAns', 'TimeAns', 'TriNoAns', 'numOfResponsesAns']



    ## Now create the difference between timeStim and TimeAns
    df_c['timeToAnswer'] = df_c['TimeAns'] - df_c['TimeStim']

    ## Now write the output
    df_c.to_csv(out_name)








222-13#_Parent_Identify.evt


/home/arosen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Final Row
250-23#_Parent_Identify_ICAb.evt
Final Row
118-229_Parent_Identify-export.evt
Final Row
118-107_Parent_Identify-export.evt
Final Row
117-76_Parent_Identify-export.evt
Final Row
248-23#_Parent_Identify_ICAb.evt
Final Row
117-32_Parent_Identify-export.evt
Final Row
250-10#_Parent_Identify.evt
Final Row
118-46#_Parent_Identify-export.evt
Final Row
255-81#_Parent_Identify_ICAb.evt
Final Row
250-22#_Parent_Identify_ICAb.evt
Final Row
118-101_Parent_Identify-export.evt
Final Row
255-64#_Parent_Identify_ICAb.evt
Final Row
118-232_Parent_Identify.evt
Final Row
250-36#_Parent_Identify_ICAb.evt
Final Row
118-198_Parent_Identify-export.evt
Final Row
118-143_Parent_Identify-export.evt
Final Row
250-26#_Parent_Identify_ICAb.evt
Final Row
117-34_Parent_Identify-export.evt
Final Row
250-39#_Parent_Identify_ICAb.evt
Final Row
117-50#_Parent_Identify-export.evt
Final Row
250-27#_Parent_Identify_ICAb.evt
Final Row
118-132_Parent_Identify-export.evt
Final Row
118-128_Parent_Identify-export.evt
